In [3]:
import pandas as pd 
import plotly.express as px 

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

import optuna

### Carga de dados

In [4]:
df_clientes = pd.read_csv('./datasets/data.csv')


In [5]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [6]:
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


### EDA

In [7]:
#distribuição da variavel inovação
percentual_inovacao = df_clientes.value_counts('inovacao') / len(df_clientes) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

In [ ]:
# Teste ANOVA analise de varianciações significativas na media de faturamento mensal para diferentes niveis de inovação
#suposições 
# as observações independentes
# variavel dependente e continua
# segue uma distribuição normal
# homogeneidade das variancias
# amostra sejam de tamanhos iguais



In [9]:
# checar se as variancias (faturamento) entre os grupos (inovação) sao homogeneas
# aplicar teste de barlett
#h0 = variancias sao igauis e h1 - variancias nao sao iguais

from scipy.stats import bartlett

#separando os dados de faturamento em grupos com base na coluna inovação
dados_agrupados = [df_clientes['faturamento_mensal'][df_clientes['inovacao'] == grupo] for grupo in df_clientes['inovacao'].unique()]

# executar o teste de bartlett
bartlett_test_statistic, bartlett_p_values = bartlett(*dados_agrupados)

#exibindo resultados
print(f'Estatistica do teste de bartlett: {bartlett_test_statistic}')
print(f'P-Value teste de bartlett: {bartlett_p_values}')

Estatistica do teste de bartlett: 10.901203117231173
P-Value teste de bartlett: 0.28254182954905804


In [11]:
# Executar o teste de Shapiro
#verificar se os dados seguem uma distribuição normal
from scipy.stats import shapiro


In [12]:

# executar o teste de shapiro
shapiro_test_statistic, shapiro_p_values = shapiro(df_clientes['faturamento_mensal'])

#exibindo resultados
print(f'Estatistica do teste de shapiro: {shapiro_test_statistic}')
print(f'P-Value teste de shapiro: {shapiro_p_values}')

Estatistica do teste de shapiro: 0.9959857602472711
P-Value teste de shapiro: 0.23513451034389005


In [13]:
# aplicar a ANOVA de welch pois as amostras sao de tamanhos diferentes
# H0 - Não ha diferenças significativas entre as médias dos grupos e H1 ha pelo menos 1

from pingouin import welch_anova

aov = welch_anova(dv='faturamento_mensal', between='inovacao', data=df_clientes)

print(f'Estatistica do teste de anova welch: {aov.loc[0, 'F']}')
print(f'P-Value teste de anova welch: {aov.loc[0, 'p-unc']}')

Estatistica do teste de anova welch: 1.1269836194061693
P-Value teste de anova welch: 0.34526211273911467


### Treinar o algoritmo kmeans

In [15]:
# selecionar as colunas para clusterização
x= df_clientes.copy()

#separar as variaveis numericas, categoricas e ordinais
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

#aplicar transformações 
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

preprocessor= ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)

    ]
)
# transformar dados
X_transformed = preprocessor.fit_transform(x)


In [16]:
X_transformed

array([[-0.74634498, -0.54179191, -1.10058849, ...,  0.        ,
         0.        ,  1.        ],
       [-0.56165548, -1.5035527 ,  1.94344851, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40582654,  1.06114274, -1.77704115, ...,  0.        ,
         0.        ,  9.        ],
       ...,
       [ 2.8196246 , -1.18296577,  0.25231684, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.03321411, -0.54179191, -1.43881482, ...,  0.        ,
         0.        ,  3.        ],
       [-2.03011486, -0.22120498, -1.77704115, ...,  1.        ,
         0.        ,  9.        ]])

In [18]:
# Optuna para otimização de hiperparams
import optuna
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances

def kmeans_objective(trial):
    # definir os hiperparâmetros a serem ajustados
    n_clusters = trial.suggest_int('n_clusters', 3, 10)
    distance_metric = trial.suggest_categorical('distance_metric', ['euclidean', 'minkowski'])

    # criar o modelo
    modelo_kmeans = KMeans(n_clusters=n_clusters, random_state=51)

    # treinar o modelo
    modelo_kmeans.fit(X_transformed)

    # calcular o silhouette score
    distances = pairwise_distances(X_transformed, metric=distance_metric)
    silhouette_avg = silhouette_score(distances, modelo_kmeans.labels_, metric='precomputed')

    return silhouette_avg


In [19]:
#criar um estudo de optuna
search_space = {'n_clusters': [3,4,5,6,7,8,9,10], 'distance_metric': ['euclidean', 'minkowski']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_kmeans = optuna.create_study(direction='maximize', sampler=sampler)

#rodar o estudo
estudo_kmeans.optimize(kmeans_objective, n_trials=100)

[I 2025-06-04 15:54:12,076] A new study created in memory with name: no-name-91578ed4-663b-4b3d-97c8-3585c3b90971
[I 2025-06-04 15:54:16,648] Trial 0 finished with value: 0.16677789435822268 and parameters: {'n_clusters': 4, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.16677789435822268.
[I 2025-06-04 15:54:16,658] Trial 1 finished with value: 0.12513401433552748 and parameters: {'n_clusters': 9, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.16677789435822268.
[I 2025-06-04 15:54:16,668] Trial 2 finished with value: 0.24380381807094345 and parameters: {'n_clusters': 3, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.24380381807094345.
[I 2025-06-04 15:54:16,681] Trial 3 finished with value: 0.16677789435822274 and parameters: {'n_clusters': 4, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.24380381807094345.
[I 2025-06-04 15:54:16,693] Trial 4 finished with value: 0.13539693514128953 and parameters: {'n_clusters': 8, 'di

In [21]:
# Melhor configuração encontrada pelo optuna
best_params = estudo_kmeans.best_params

# Instanciando kmeans com melhores parâmetros
best_kmeans = KMeans(n_clusters=best_params['n_clusters'], random_state=51)
best_kmeans.fit(X_transformed)

# Calcular o silhouette score
distances = pairwise_distances(X_transformed, metric=best_params['distance_metric'])
best_silhouette = silhouette_score(distances, best_kmeans.labels_)

# Impressão dos resultados
print(f'k (número de clusters): {best_params["n_clusters"]}')
print(f'Métrica de distância selecionada: {best_params["distance_metric"]}')
print(f'Melhor Silhouette Score: {best_silhouette}')


k (número de clusters): 3
Métrica de distância selecionada: minkowski
Melhor Silhouette Score: 0.44454582909990875


In [22]:
#criar coluna com cluster escolhido
df_clientes['cluster'] = best_kmeans.labels_

In [23]:
#visualizar
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


### Visualizar resultados

In [24]:
#cruzar idade e faturamento apresentando clusters
px.scatter(df_clientes, x='idade', y='faturamento_mensal', color='cluster')

In [25]:
#cruzar idade e faturamento apresentando clusters
px.scatter(df_clientes, x='inovacao', y='faturamento_mensal', color='cluster')

In [27]:
#cruzar idade e faturamento apresentando clusters
px.scatter(df_clientes, x='numero_de_funcionarios', y='faturamento_mensal', color='cluster')

In [28]:
#salvar modelo e pipeline
import joblib

joblib.dump(best_kmeans, 'modelo_clusterizacao.pkl')

#salvar pipeline
joblib.dump(preprocessor, 'pipeline_cluterizacao.pkl')

['pipeline_cluterizacao.pkl']

### Aplicação batch no gradio

In [29]:
import gradio as gr
modelo = joblib.load('./modelo_clusterizacao.pkl')
preprocessor = joblib.load('./pipeline_cluterizacao.pkl')

def clustering(arquivo):
    #Carregar o CSV em um DataFrame
    df_empresas = pd.read_csv(arquivo.name)

    #transformar os dados do df para o formato que o kmeans aceita
    X_transformed = preprocessor.fit_transform(df_empresas)

    #Treinar modelo
    modelo.fit(X_transformed)

    #Criar a coluna cluster no df
    df_empresas['cluster'] = modelo.labels_
    df_empresas.to_csv('./clusters.csv', index=False)

    return './clusters.csv'




In [31]:
#Criar interface
app = gr.Interface(
    clustering,
    gr.File(file_types=[".csv"]),
    "file"
)

#rodar app
app.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv
